In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
import time
import math

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
## Specify appropriate transforms, and batch_sizes
train_dir = '../05_Final_Landmark_Identification/landmark_images/train'
test_dir = '../05_Final_Landmark_Identification/landmark_images/test'

batch_size = 50
valid_size = 0.2

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
normalization = transforms.Normalize(mean=mean, std=std)

data_transforms = {
  'augment': transforms.Compose([transforms.RandomRotation(30),
                              transforms.RandomResizedCrop(224),
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
                              normalization,]),
  'no_augment' : transforms.Compose([transforms.Resize(224),
                              transforms.CenterCrop(224),
                              transforms.ToTensor(),
                              normalization,])
}

image_datasets = {
  'train': datasets.ImageFolder(train_dir, transform=data_transforms['augment']),
  'valid': datasets.ImageFolder(train_dir, transform=data_transforms['no_augment']),
  'test' : datasets.ImageFolder(test_dir, transform=data_transforms['no_augment'])
}

num_train = len(image_datasets['train'])
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

label_mapping = image_datasets['train'].classes
label_mapping = list(map(lambda x: x.split('.')[1].replace('_', ' '), label_mapping))
print(len(label_mapping), 'classes in this dataset')
print(label_mapping)

In [ ]:
loaders_transfer = {
  'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, sampler=train_sampler),
  'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=batch_size, sampler=valid_sampler),
  'test' : torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=True),
}

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show5(dataloader, label_mapping):
    dataiter = iter(dataloader)
    fig, ax = plt.subplots(figsize=(18, 10), ncols=5)

    batch = next(dataiter)
    labels = batch[1][0:5]
    images = batch[0][0:5]
    for i in range(5):
        title = label_mapping[labels[i].item()]
        print(f'Class: {title}, Shape: {images[i].shape}')
    
        image = images[i].numpy().T
        image = std * image + mean
        image = np.clip(image, 0, 1)
        ax[i].set_title(title)
        ax[i].imshow(image)


show5(loaders_transfer['train'], label_mapping)


In [ ]:
# useful variable that tells us whether we should use the GPU
use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for data, target in loaders['test']:
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            # forward pass: compute predicted outputs by passing inputs to the model
            log_ps = model.forward(data)
            ps = torch.exp(log_ps)
            # calculate the loss
            loss = criterion(log_ps, target)
            # update average test loss 
            test_loss += loss.item()
            # convert output probabilities to predicted class
            top_p, top_class = ps.topk(1, dim=1)
            # compare predictions to true label
            correct += torch.sum((top_class == target.reshape(*top_class.shape)).type(torch.FloatTensor)).item()
            total += data.size(0)

    
    test_loss = test_loss/len(loaders['test'])
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
criterion_transfer = nn.NLLLoss()

def get_optimizer_transfer(model):
    return optim.Adam(model.parameters())    

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    valid_loss_min = np.Inf 
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            log_ps = model.forward(data)
            loss = criterion(log_ps, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()          

        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                log_ps = model.forward(data)
                loss = criterion(log_ps, target)
                valid_loss += loss.item()            

        train_loss = train_loss/len(loaders['train'])
        valid_loss = valid_loss/len(loaders['valid'])

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        if valid_loss < valid_loss_min:
            print('Validation loss has decreased, Saving...')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
        
    return model

In [ ]:
model_transfer = torchvision.models.vgg16(pretrained=True)
for param in model_transfer.parameters():
    param.requires_grad = False

print('model\'s classifier expects', model_transfer.classifier[0].in_features, 'in_features')

model_transfer.classifier = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(25088, 5000)),
                ('relu1', nn.ReLU()),
                ('dropout1', nn.Dropout(p=0.2)),
                ('fc2', nn.Linear(5000, 1000)),
                ('relu2', nn.ReLU()),
                ('dropout2', nn.Dropout(p=0.2)),
                ('fc3', nn.Linear(1000, 500)),
                ('relu3', nn.ReLU()),
                ('dropout3', nn.Dropout(p=0.2)),
                ('fc4', nn.Linear(500, 100)),
                ('relu4', nn.ReLU()),
                ('dropout4', nn.Dropout(p=0.2)),
                ('fc_final', nn.Linear(100, 50)),
                ('log_output', nn.Softmax(dim=1))
            ]))

#-#-# Do NOT modify the code below this line. #-#-#

if use_cuda:
    model_transfer = model_transfer.cuda()

In [ ]:

# num_epochs = 50
# model_transfer = train(num_epochs, loaders_transfer, model_transfer, get_optimizer_transfer(model_transfer), 
#                       criterion_transfer, use_cuda, 'model_transfer.pt')


In [ ]:
model_transfer.load_state_dict(torch.load('../05_Final_Landmark_Identification/model_transfer.pt'))

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

In [ ]:
def im_convert(tensor):
    """ Display a tensor as an image. """
    
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

In [ ]:
target_image = np.random.randint(0, high=1, size=(3,224,224))
target = torch.from_numpy(target_image).type(torch.FloatTensor).unsqueeze(0).cuda().requires_grad_(True)
plt.imshow(im_convert(target))

expected = np.zeros((50,))
expected[0] = 1.
expected = torch.from_numpy(expected).cuda()
expected.requires_grad = False


In [ ]:
for param in model_transfer.parameters():
    param.requires_grad = False

optimizer = optim.Adam([target], lr=0.01)
steps = 10000
show_every = 100

model_transfer.eval()

for ii in range(steps):
    output = model_transfer.forward(target)
    loss = torch.mean((output - expected)**2).requires_grad_(True)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if  ii % show_every == 0:
        print('loss: ', loss.item())
        plt.imshow(im_convert(target))
        plt.show()



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(im_convert(target_image))
ax2.imshow(im_convert(target))
plt.show()